## Importing libraries and modules

In [1]:
# Enable auto-reload
%load_ext autoreload
%autoreload 2

In [2]:
# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Add one level up directory to the path
import sys
sys.path.append("..")

# Import libraries
import torch
import random
import pandas as pd

# Import custom modules
from src.data import *
from src.machine_translation import *
from src.data.utils import get_dataset
from src.data.preprocess import clean_text
from src.data.tokenizer import CustomBartTokenizer
from src.machine_translation.translate import translate
from src.machine_translation.net import CodeMixedModel, CodeMixedModelHGTrainer
from src.machine_translation.models.bart_conditional import BartForConditionalGeneration
from src.machine_translation.utils import get_tokenized_dataset_models, get_data_loader_models, calculate_sacrebleu_score, calculate_chrf_score, calculate_bert_score, calculate_tokens

## Data acquisition, cleaning and processing

In [3]:
%%time
train_df, validation_df, test_df = get_dataset()

CPU times: user 193 ms, sys: 18.8 ms, total: 212 ms
Wall time: 231 ms


In [4]:
train_df

,hi_en,en,source
0,film ka kya naam hai,What's the name of the movie,cmu_hinglish_dog
1,"namaste, sada hua tomatoes score mahaan hai, l...","Hi, the rotten tomatoes score is great but the...",cmu_hinglish_dog
2,kya aapako lagata hai ki aapako film pasand aa...,Do you think you will like the movie,cmu_hinglish_dog
3,yah kis tarah kee philm hai,What kind of movie is it,cmu_hinglish_dog
4,film kab banee thee?,when was the movie made?,cmu_hinglish_dog
...,...,...,...
174438,Thik hai\n,Ok.,linc
174439,Thik hai bhai\n,ok bro,linc
174440,Kya ham chalu kar sakte hai?\n,shall we continue?,linc
174441,Kya aapko pasand hai hamare saath\n,do you like we can,linc


In [5]:
validation_df

,hi_en,en,source
0,movie kis baare me hai?,What is the movie about?,cmu_hinglish_dog
1,Movie ek chhote bacche Kevin k baare me hai ji...,the movie is about a young child named Kevin w...,cmu_hinglish_dog
2,Kya wo jaan bhuj k abandon karte hai?,Did they abandon him on purpose?,cmu_hinglish_dog
3,"nahi, wo uska track lose kardete hai kyuki bah...",no they had lost track of him since they had m...,cmu_hinglish_dog
4,Kya wo realize karte hai k wo chhut gaya aur u...,Did they realize they lost track of him and co...,cmu_hinglish_dog
...,...,...,...
3238,Sunkar good movie hai. Kya ham finish kar skat...,Sounds like a good movie. Can we finish now?,linc
3239,Yep. Thanks baat karne ke liye\n,Yep. Thanks for chatting,linc
3240,"Thanks, mei dekhati hui. Achi baat hai\n","thanks, I will watch it. SOunds good",linc
3241,kya tumhe movie Despicable Me pasand hai?\n,Did you like the movie Despicable Me?,linc


In [6]:
test_df

,hi_en,en,source
0,oye ledki kitni mazakiya movie dekh rhi ho ?,ah mean girls. such a funny movie. have you se...,cmu_hinglish_dog
1,ha muje bhi hassi mazak ki movies pasand par y...,"Yeah, even though I love comedies, this wasn't...",cmu_hinglish_dog
2,kese tina ka jhuti writing isme achi the usne ...,how come. thought tina fey's writing was fanta...,cmu_hinglish_dog
3,muje kahani bhute achi lagi aur unhone ise dac...,I loved the story & how true they made it in h...,cmu_hinglish_dog
4,ab samja muje nhi pta tha ki aap sahi admi hai...,I see. i didn't realize it was 14 years ago. y...,cmu_hinglish_dog
...,...,...,...
7067,alarm ko abhi stop kare,Stop alarm now,top
7068,Har ghante ke liye alarm set kare,set alarm every hour,top
7069,Bobby ko text kare,text Bobby,top
7070,Muje shaam 6 baje laundry ko pick up karne ke ...,remind me to pick up laundry at 6 pm,top


In [7]:
%%time
# Cleaning train_df
train_df = train_df.applymap(clean_text)
train_df

CPU times: user 2.86 s, sys: 10.4 ms, total: 2.87 s
Wall time: 2.87 s


,hi_en,en,source
0,film ka kya naam hai,what's the name of the movie,cmu_hinglish_dog
1,"namaste, sada hua tomatoes score mahaan hai, l...","hi, the rotten tomatoes score is great but the...",cmu_hinglish_dog
2,kya aapako lagata hai ki aapako film pasand aa...,do you think you will like the movie,cmu_hinglish_dog
3,yah kis tarah kee philm hai,what kind of movie is it,cmu_hinglish_dog
4,film kab banee thee?,when was the movie made?,cmu_hinglish_dog
...,...,...,...
174438,thik hai,ok.,linc
174439,thik hai bhai,ok bro,linc
174440,kya ham chalu kar sakte hai?,shall we continue?,linc
174441,kya aapko pasand hai hamare saath,do you like we can,linc


In [8]:
%%time
# Cleaning validation_df
validation_df = validation_df.applymap(clean_text)
validation_df

CPU times: user 53.3 ms, sys: 937 µs, total: 54.2 ms
Wall time: 53.8 ms


,hi_en,en,source
0,movie kis baare me hai?,what is the movie about?,cmu_hinglish_dog
1,movie ek chhote bacche kevin k baare me hai ji...,the movie is about a young child named kevin w...,cmu_hinglish_dog
2,kya wo jaan bhuj k abandon karte hai?,did they abandon him on purpose?,cmu_hinglish_dog
3,"nahi, wo uska track lose kardete hai kyuki bah...",no they had lost track of him since they had m...,cmu_hinglish_dog
4,kya wo realize karte hai k wo chhut gaya aur u...,did they realize they lost track of him and co...,cmu_hinglish_dog
...,...,...,...
3238,sunkar good movie hai. kya ham finish kar skat...,sounds like a good movie. can we finish now?,linc
3239,yep. thanks baat karne ke liye,yep. thanks for chatting,linc
3240,"thanks, mei dekhati hui. achi baat hai","thanks, i will watch it. sounds good",linc
3241,kya tumhe movie despicable me pasand hai?,did you like the movie despicable me?,linc


In [9]:
%%time
# Cleaning test_df
test_df = test_df.applymap(clean_text)
test_df

CPU times: user 107 ms, sys: 938 µs, total: 108 ms
Wall time: 107 ms


,hi_en,en,source
0,oye ledki kitni mazakiya movie dekh rhi ho ?,ah mean girls. such a funny movie. have you se...,cmu_hinglish_dog
1,ha muje bhi hassi mazak ki movies pasand par y...,"yeah, even though i love comedies, this wasn't...",cmu_hinglish_dog
2,kese tina ka jhuti writing isme achi the usne ...,how come. thought tina fey's writing was fanta...,cmu_hinglish_dog
3,muje kahani bhute achi lagi aur unhone ise dac...,i loved the story & how true they made it in h...,cmu_hinglish_dog
4,ab samja muje nhi pta tha ki aap sahi admi hai...,i see. i didn't realize it was 14 years ago. y...,cmu_hinglish_dog
...,...,...,...
7067,alarm ko abhi stop kare,stop alarm now,top
7068,har ghante ke liye alarm set kare,set alarm every hour,top
7069,bobby ko text kare,text bobby,top
7070,muje shaam 6 baje laundry ko pick up karne ke ...,remind me to pick up laundry at 6 pm,top


## Data tokenization

In [10]:
%%time
bart_tokenizer = CustomBartTokenizer().build()
bart_tokenizer_scratch = CustomBartTokenizer().build(data=train_df["hi_en"], tokenizer_style=STYLE.SCRATCH.value)
bart_tokenizer_append = CustomBartTokenizer().build(data=train_df["hi_en"], tokenizer_style=STYLE.APPEND.value)







CPU times: user 27.5 s, sys: 113 ms, total: 27.7 s
Wall time: 48.5 s


In [11]:
query = "Hello! Ye CSCI 544 ka project he. Ye project Code-Mixed Machine Translation ke bara me he. Hamane hamari khoon bahahe."
query = clean_text(text=query)

In [12]:
token_idx = bart_tokenizer.encode(query, add_special_tokens=True)
token_word = bart_tokenizer.convert_ids_to_tokens(token_idx)
print('#'*100)
print("This is an example of using the default Bart Tokenizer.\n")
print(f"Sentence: {query}\n")
print(f"Tokenized word: {token_word}\n")
print(f"Tokenized idx: {token_idx}")
print('#'*100)

####################################################################################################
This is an example of using the default Bart Tokenizer.

Sentence: hello! ye csci 544 ka project he. ye project code-mixed machine translation ke bara me he. hamane hamari khoon bahahe.

Tokenized word: ['<s>', 'hello', '!', 'Ġye', 'Ġc', 'sci', 'Ġ5', '44', 'Ġka', 'Ġproject', 'Ġhe', '.', 'Ġye', 'Ġproject', 'Ġcode', '-', 'm', 'ixed', 'Ġmachine', 'Ġtranslation', 'Ġke', 'Ġbar', 'a', 'Ġme', 'Ġhe', '.', 'Ġham', 'ane', 'Ġham', 'ari', 'Ġkh', 'oon', 'Ġb', 'aha', 'he', '.', '</s>']

Tokenized idx: [0, 42891, 328, 32440, 740, 43428, 195, 3305, 4661, 695, 37, 4, 32440, 695, 3260, 12, 119, 24194, 3563, 19850, 7321, 2003, 102, 162, 37, 4, 11402, 1728, 11402, 1512, 16447, 3863, 741, 11695, 700, 4, 2]
####################################################################################################


In [13]:
token_idx = bart_tokenizer_scratch.encode(query, add_special_tokens=True)
token_word = bart_tokenizer_scratch.convert_ids_to_tokens(token_idx)
print('#'*100)
print("This is an example of the Bart Tokenizer that is trained from scratch.\n")
print(f"Sentence: {query}\n")
print(f"Tokenized word: {token_word}\n")
print(f"Tokenized idx: {token_idx}")
print('#'*100)

####################################################################################################
This is an example of the Bart Tokenizer that is trained from scratch.

Sentence: hello! ye csci 544 ka project he. ye project code-mixed machine translation ke bara me he. hamane hamari khoon bahahe.

Tokenized word: ['<s>', 'hello', '!', 'Ġye', 'Ġc', 'sci', 'Ġ5', '44', 'Ġka', 'Ġproject', 'Ġhe', '.', 'Ġye', 'Ġproject', 'Ġcode', '-', 'mix', 'ed', 'Ġmachine', 'Ġtrans', 'lation', 'Ġke', 'Ġbara', 'Ġme', 'Ġhe', '.', 'Ġhamane', 'Ġhamari', 'Ġkhoon', 'Ġbaha', 'he', '.', '</s>']

Tokenized idx: [0, 2317, 4, 520, 288, 21554, 466, 11342, 312, 3070, 356, 17, 520, 3070, 6438, 16, 19530, 517, 6262, 3397, 18402, 271, 17190, 284, 356, 17, 11714, 4630, 10623, 13967, 282, 17, 2]
####################################################################################################


In [14]:
token_idx = bart_tokenizer_append.encode(query, add_special_tokens=True)
token_word = bart_tokenizer_append.convert_ids_to_tokens(token_idx)
print('#'*100)
print("This is an example of the Bart Tokenizer to which we append our new data.\n")
print(f"Sentence: {query}\n")
print(f"Tokenized word: {token_word}\n")
print(f"Tokenized idx: {token_idx}")
print('#'*100)

####################################################################################################
This is an example of the Bart Tokenizer to which we append our new data.

Sentence: hello! ye csci 544 ka project he. ye project code-mixed machine translation ke bara me he. hamane hamari khoon bahahe.

Tokenized word: ['<s>', 'hello', '!', 'Ġ', 'ye', 'Ġ', 'csc', 'i', 'Ġ', '54', '4', 'Ġ', 'ka', 'Ġ', 'project', 'Ġ', 'he', '.', 'Ġ', 'ye', 'Ġ', 'project', 'Ġ', 'code', '-', 'mix', 'ed', 'Ġ', 'mach', 'ine', 'Ġ', 'trans', 'lation', 'Ġ', 'ke', 'Ġ', 'bara', 'Ġ', 'me', 'Ġ', 'he', '.', 'Ġ', 'hamane', 'Ġ', 'hamari', 'Ġ', 'khoo', 'n', 'Ġ', 'baha', 'he', '.', '</s>']

Tokenized idx: [0, 42891, 328, 1437, 4717, 1437, 82131, 118, 1437, 4283, 306, 1437, 2348, 1437, 28258, 1437, 700, 4, 1437, 4717, 1437, 28258, 1437, 20414, 12, 39915, 196, 1437, 79389, 833, 1437, 9981, 35019, 1437, 1071, 1437, 31533, 1437, 1794, 1437, 700, 4, 1437, 77062, 1437, 73169, 1437, 64064, 282, 1437, 53028, 700, 4, 2]
########

In [15]:
%%time
# Now that we know how to get a BART-Tokenizer - default, appeded and scratch, let us get the tokenizer for our code-mixed language
# and the target language.
# Code-Mixed Language - Build a Bart-Tokenizer from scratch.
# Target Language (English) - Use the default Bart-Tokenizer.
hi_en_bart_tokenizer = CustomBartTokenizer().build(
    data=train_df["hi_en"],
    tokenizer_style=MBART_TOKENIZER_BPE_BINDING_BART_TOKENIZER_ENCODER_STYLE,
    tokenizer_bart_from_pretrained_path=MBART_TOKENIZER_BPE_BINDING_BART_TOKENIZER_ENCODER_FROM_PRETRAINED
)
en_bart_tokenizer = CustomBartTokenizer().build(
    tokenizer_style=MBART_TOKENIZER_BPE_BINDING_BART_TOKENIZER_DECODER_STYLE,
    tokenizer_bart_from_pretrained_path=MBART_TOKENIZER_BPE_BINDING_BART_TOKENIZER_DECODER_FROM_PRETRAINED
)




CPU times: user 13.5 s, sys: 38.8 ms, total: 13.6 s
Wall time: 13.8 s


In [16]:
print(f"Hinglish tokenizer vocab size: {hi_en_bart_tokenizer.vocab_size}")
print(f"English tokenizer vocab size: {en_bart_tokenizer.vocab_size}")

Hinglish tokenizer vocab size: 50265
English tokenizer vocab size: 50265


In [17]:
%%time
print(f"There are {train_df['hi_en'].apply(lambda x: calculate_tokens(x, hi_en_bart_tokenizer)).sum()} hinglish tokens in the train data.")
print(f"There are {train_df['en'].apply(lambda x: calculate_tokens(x, en_bart_tokenizer)).sum()} english tokens in the train data.")
print(f"There are {validation_df['hi_en'].apply(lambda x: calculate_tokens(x, hi_en_bart_tokenizer)).sum()} hinglish tokens in the validation data.")
print(f"There are {validation_df['en'].apply(lambda x: calculate_tokens(x, en_bart_tokenizer)).sum()} english tokens in the validation data.")
print(f"There are {test_df['hi_en'].apply(lambda x: calculate_tokens(x, hi_en_bart_tokenizer)).sum()} hinglish tokens in the test data.")
print(f"There are {test_df['en'].apply(lambda x: calculate_tokens(x, en_bart_tokenizer)).sum()} english tokens in the test data.")

There are 1764738 hinglish tokens in the train data.
There are 1807859 english tokens in the train data.
There are 38502 hinglish tokens in the validation data.
There are 38366 english tokens in the validation data.
There are 70895 hinglish tokens in the test data.
There are 69686 english tokens in the test data.
CPU times: user 9.23 s, sys: 11.4 ms, total: 9.24 s
Wall time: 9.28 s


## Dataset and Data Loaders

In [18]:
# Get the tokenized dataset
__tokenized_datasets__ = get_tokenized_dataset_models(
    train_df=train_df,
    validation_df=validation_df,
    test_df=test_df,
    encoder_tokenizer=hi_en_bart_tokenizer,
    decoder_tokenizer=en_bart_tokenizer
)
__tokenized_datasets__

defaultdict(dict,
            {'hi_en__en': {'train': <src.machine_translation.data.CodeMixedTokenizedDataset at 0x17f2ae4c0>,
              'validation': <src.machine_translation.data.CodeMixedTokenizedDataset at 0x17f2aeee0>,
              'test': <src.machine_translation.data.CodeMixedTokenizedDataset at 0x17f2aebe0>}})

In [19]:
__train_dataset__, __validation_dataset__, __test_dataset__ = __tokenized_datasets__["hi_en__en"].values()

In [20]:
__train_dataset__.visualize()

####################################################################################################
Dataset
Number of instances:  174443
Denoising stage:  False
Source language:  hi_en
Target language:  en
Key: input_ids, Value: tensor([    0,  3497,    15, 10386,  1071,  2025,  2185,  6089,   278,    15,
         1088,  4175,  2430,  2185,   392, 33673,  1711,  1963,   291,  1104,
         1067,  1758,   278,    17,     2,     1,     1,     1,     1,     1]), Value shape: torch.Size([30])
Key: attention_mask, Value: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 0, 0]), Value shape: torch.Size([30])
Key: labels, Value: tensor([    0,  3592,     6,     5, 34485, 18553,  1471,    16,   372,    53,
            5, 32820,  7745,  1471,  1302,    10,   410,   614,    10,  1569,
            9,    42,  1318,     4,     2,  -100,  -100,  -100,  -100,  -100]), Value shape: torch.Size([30])
####################################################

In [21]:
# Get the data loaders for denoising
__data_loaders__ = get_data_loader_models(
    train_df=train_df,
    validation_df=validation_df,
    test_df=test_df,
    train_batch_size=8, 
    validation_batch_size=8, 
    test_batch_size=8,
    encoder_tokenizer=hi_en_bart_tokenizer,
    decoder_tokenizer=en_bart_tokenizer, # Get overwritten in the denoising stage
    denoising_stage=True
)

In [22]:
__data_loaders__

defaultdict(dict,
            {'hi_en__en': {'object': <src.machine_translation.data.CodeMixedDataLoader at 0x2a0ebb5e0>,
              'train': <torch.utils.data.dataloader.DataLoader at 0x2a0ebbe80>,
              'validation': <torch.utils.data.dataloader.DataLoader at 0x2a0ebba00>,
              'test': <torch.utils.data.dataloader.DataLoader at 0x2a0ebb760>}})

In [23]:
__data_loader__, __train_data_loader__, __validation_data_loader__, __test_data_loader__ = __data_loaders__["hi_en__en"].values()

In [24]:
__data_loader__.visualize()

####################################################################################################
Train Dataloader
Batch Size:  8
Number of batches:  21806
Batch source language shape:  torch.Size([8, 30])
Batch source language:  ['kya mere ghar ko koi dusra raasta hai', 'monday aur tuesday ke liye alarms create karo', 'mujhe kitne bajhe nikalna chahiye 5 pm tak new york city pahuchne ke liye', 'alarm abhi mute kare', 'connecticut se rhode island tak kitne miles hai', 'mujhe kal make up karne ke liye yaad dilaye', '5 pm ko dinner lene ke liye reminder set kare', 'kya aj mujhe scarf pehenna chahiye ?']
Batch source tokens:  tensor([[    0,   316,   376,   527,   280,   418, 50265,  1557,   278,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [    0,   969,   368,   762,   271,   283,   512,   768,   371,     2,
             1,     1,     1,     1,     1,     1

100%|███████████| 21806/21806 [00:43<00:00, 505.01it/s]


Validation of train loader successful.
####################################################################################################
Val Dataloader
Batch Size:  8
Number of batches:  406
Validating validation laoder...


100%|████████████████| 406/406 [00:33<00:00, 12.05it/s]


Validation of validation loader successful.
####################################################################################################
Test Dataloader
Batch Size:  8
Number of batches:  884
Validating test laoder...


100%|████████████████| 884/884 [00:33<00:00, 26.16it/s]

Validation of test loader successful.
####################################################################################################


## Model definition

In [25]:
__model__ = BartForConditionalGeneration()
__model__.model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

In [26]:
__batch_size__ = 8
__en_seq_length__ = random.randint(13, 23)
__de_seq_length__ = random.randint(13, 23)
__encoder_input__ = torch.randint(low=0, high=hi_en_bart_tokenizer.vocab_size, size=(__batch_size__, __en_seq_length__))
__decoder_input__ = torch.randint(low=0, high=en_bart_tokenizer.vocab_size, size=(__batch_size__, __de_seq_length__))
__encoder_input__ = __encoder_input__.to(MBART_MODEL_CONDITIONAL_GENERATION_DEVICE)
__decoder_input__ = __decoder_input__.to(MBART_MODEL_CONDITIONAL_GENERATION_DEVICE)
__out__ = __model__.model(input_ids=__encoder_input__, decoder_input_ids=__decoder_input__, return_dict=True)
print(f"Model has a total of {__model__.model.num_parameters()} number of parameters")
print("Model encoder input size: ", __encoder_input__.size())
print("Model decoder input size: ", __decoder_input__.size())
print("Model output type: ", type(__out__))
print("Model output size: ", __out__.logits.size())
print("Model output:\n", __out__.logits)

Model has a total of 406291456 number of parameters
Model encoder input size:  torch.Size([8, 23])
Model decoder input size:  torch.Size([8, 13])
Model output type:  <class 'transformers.modeling_outputs.Seq2SeqLMOutput'>
Model output size:  torch.Size([8, 13, 50265])
Model output:
 tensor([[[ 1.3886e+01, -1.2782e+00,  5.2260e+00,  ...,  2.7985e-02,
          -4.2849e-01,  3.5954e+00],
         [-1.4220e+01, -2.2313e+00,  6.5215e+00,  ..., -1.2482e+00,
          -5.2519e-01,  7.6988e-01],
         [-1.2062e+01, -3.0549e+00,  3.1566e+00,  ..., -1.6747e+00,
          -1.9740e+00, -4.2042e-01],
         ...,
         [-8.0082e+00, -3.0474e+00,  3.5415e+00,  ..., -2.0588e+00,
          -2.7357e+00, -6.6309e-01],
         [-9.0567e+00, -2.9443e+00,  4.4974e+00,  ..., -2.2417e+00,
          -3.0491e+00, -3.3858e-01],
         [-7.9153e+00, -2.8651e+00,  4.2635e+00,  ..., -1.6429e+00,
          -2.6986e+00, -6.3881e-01]],

        [[ 1.8973e+01, -1.2546e+00,  9.0026e+00,  ..., -5.0097e-02,
  

## Model training

In [27]:
# Get the tokenized dataset
dataset = get_tokenized_dataset_models(
    train_df=train_df,
    validation_df=validation_df,
    test_df=test_df,
    encoder_tokenizer=hi_en_bart_tokenizer,
    decoder_tokenizer=en_bart_tokenizer,
    denoising_stage=True
)
train_dataset, validation_dataset, test_dataset = dataset[MBART_DATALOADER_TRANSLATION_MODE].values()

In [30]:
train_dataset.visualize()

####################################################################################################
Dataset
Number of instances:  174443
Denoising stage:  True
Source language:  hi_en
Target language:  hi_en
Key: input_ids, Value: tensor([    0,  3497,    15, 10386,  1071,  2025,  2185,  6089,   278,    15,
         1088,  4175,  2430,  2185,   392, 33673,  1711,  1963,   291,  1104,
         1067,  1758,   278,    17,     2,     1,     1,     1,     1,     1]), Value shape: torch.Size([30])
Key: attention_mask, Value: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 0, 0]), Value shape: torch.Size([30])
Key: labels, Value: tensor([    0,  3497,    15, 10386,  1071,  2025,  2185,  6089,   278,    15,
         1088,  4175,  2430,  2185,   392, 33673,  1711,  1963,   291,  1104,
         1067,  1758,   278,    17,     2,  -100,  -100,  -100,  -100,  -100]), Value shape: torch.Size([30])
##################################################

In [28]:
# Initalize the model
mt_hg_model = CodeMixedModelHGTrainer(
    train_dataset = train_dataset,
    validation_dataset = validation_dataset,
    test_dataset = test_dataset,
    encoder_tokenizer=hi_en_bart_tokenizer,
    decoder_tokenizer=en_bart_tokenizer,
    denoising_stage=True
)

Getting the model...

Configuring optimizer and scheduler...

Configuring collator...

Configuring trainer...

Configuring training arguments...


In [ ]:
%%time
# Train the model
mt_hg_model.fit(skip_training=False)

In [ ]:
%%time
# Evaluate model on test data
mt_hg_model.predict(model_path=MBART_MODEL_CONDITIONAL_GENERATION_RESUME_FROM_CHECKPOINT)

In [ ]:
# Run inference
src = "film <mask> kya <mask> hai?"
print(f"\nSRC: {src}\nTGT: {mt_hg_model.infer(model_path=MBART_MODEL_CONDITIONAL_GENERATION_RESUME_FROM_CHECKPOINT, src=src)}")